In [11]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import microatoll_sim.simulator as sim
import microatoll_sim.analytical_simulator as asim

## Setup


In [2]:
params = sim.SimParams(
    vert_shift=0.4,
    gr_mean=15,
    d=1.0,
    dt=0.1, # will have 1/dt = 10 bands/yr
    T0=1980,
    delta_T=30,
    init_radius=0.3
)


sl_df = pd.read_csv("./data/SeaLevel.csv", header=None, names=["time", "sl"])
sl_arr = sl_df.to_numpy()
sl_arr[:, 1] += params.vert_shift
band_sl_arr = sim.lowest_discreet(sl_arr, params.dt, params.T0, params.T0+params.delta_T)
gr_vec = np.ones(params.NT) * params.gr_mean

## Iterative simulator benchmarks


In [4]:
%timeit sim.coral_growth_all(params.init_radius, params.num_initial_pts, params.d, gr_vec, params.NT, band_sl_arr[:, 1])

174 ms ± 771 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%timeit sim.lowest_discreet(sl_arr, params.dt, params.T0, params.T0+params.delta_T)

75 µs ± 626 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
lines, living_statuses = sim.coral_growth_all(
    params.init_radius, params.num_initial_pts, params.d, gr_vec, params.NT, band_sl_arr[:, 1]
)
line = lines[-1]
living_status = living_statuses[-1]

In [8]:
%timeit sim.resample(line, living_status,params.d)

33.8 µs ± 3.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
%timeit sim.kill_loop(line, living_status,params.gr_mean) 

2.51 ms ± 33.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit sim.get_pointwise_unit_normals(line)

3.79 µs ± 6.85 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Analytical simulator benchmarks
